In [36]:
import sqlite3
import os
import pysolr

sqlitedb = os.path.join(os.path.expanduser('~'),'Box Sync', 'GradSchoolStuff', 'MastersProject', 'mimic3', 'mimic3.sqlite')
if not (os.path.exists(sqlitedb)):
    print("Specified database does not exist")
    sys.exit()

connection = sqlite3.connect(sqlitedb)

# Connect to Solr instance.
solr = pysolr.Solr('http://192.168.99.100:8983/solr/radiology')

with connection:
    cur = connection.cursor()
    recordset = cur.execute("select * from noteevents where description like '%CT%' and category = 'Radiology' order by row_id")
    col_names = [cn[0] for cn in cur.description]

    for row in recordset:
        document = {}
        #print('++++++', col_names[idx], ': ', col)
        # want row_id, subject_id, hadm_id, description, text
        document['id'] = row[col_names.index("ROW_ID")]
        document['SUBJECT_ID_i'] = row[col_names.index("SUBJECT_ID")]
        # not all reports have a hospital admission id, so this may not be in Solr
        document['HADM_ID_i'] = row[col_names.index("HADM_ID")]
        # store description as a single string, but also allow for searching parts
        document['DESCRIPTION_s'] = row[col_names.index("DESCRIPTION")]
        document['DESCRIPTION_t'] = row[col_names.index("DESCRIPTION")]
        document['TEXT_t'] = row[col_names.index("TEXT")]
        
        print('Adding row_id', row[col_names.index("ROW_ID")], 'to Solr.')
        solr.add([document])

Adding row_id 733930 to Solr.
Adding row_id 733931 to Solr.
Adding row_id 733935 to Solr.
Adding row_id 733957 to Solr.
Adding row_id 733971 to Solr.
